## Import Packages

In [ ]:
import albumentations
import albumentations.pytorch
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm.notebook import tqdm

import timm
import ttach as tta
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from sklearn.metrics import f1_score
from efficientnet_pytorch import EfficientNet

tqdm.pandas()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"{device} is using!")

## Config Setting

In [ ]:
config = {'NUM_EPOCHS' : 15,
          'BATCH_SIZE' : 8,
          'NUM_CLASSES' : 18,
          'LEARNING_RATE' : 1e-4,
          'MODEL' : 'nf_resnet50',  # 불러올 모델 이름
          'MODEL_NAME' : 'nf_resnet50',  # 저장할 떄 이름
          'NUM_WORKERS' : 2,
          'LOG_STEPS' : 450,
          'SAVE_PATH' : './epoch/',
          'LOAD_MODEL' : False,  # 학습을 이어서 할 때 True
          'LOAD_MODEL_PATH' : './epoch/nf_resnet50_epoch_4_0.7522671719026435.pth'  # 이어서 학습할 파일의 경로
         }

## Augmentation Setting

In [ ]:
train_transform = albumentations.Compose(
  [
      albumentations.Resize(256,256),
#       albumentations.RandomRotation(15),
#       albumentations.HorizontalFlip(p=0.3),
      albumentations.OneOf([albumentations.ShiftScaleRotate(rotate_limit=15, p=0.5),
                            albumentations.RandomBrightnessContrast(p=0.5),
                            albumentations.MotionBlur(p=0.5),
                            albumentations.OpticalDistortion(p=0.5),
                            albumentations.GaussNoise(p=0.5)], p=1),
      albumentations.Normalize((0.548, 0.504, 0.479), (0.237, 0.247, 0.246)),
      albumentations.pytorch.transforms.ToTensorV2(),
      #       이미지 원본 사이즈는 384, 512   
  ]
)

test_transform = albumentations.Compose(
  [
      albumentations.Resize(288,288),
      albumentations.Normalize((0.548, 0.504, 0.479), (0.237, 0.247, 0.246)),
      albumentations.pytorch.transforms.ToTensorV2()
      #       이미지 원본 사이즈는 384, 512   
  ]
)

## Read DataFrame

In [ ]:
def make_test_full_path(s):
    path = 'input/data/eval/images/'
    return path + s

train_df = pd.read_csv('./stratified_df/train_df.csv')
valid_df = pd.read_csv('./stratified_df/valid_df.csv')
test_df = pd.read_csv('./input/data/eval/info.csv')
test_df['full_path'] = test_df['ImageID'].progress_apply(make_test_full_path)
submission_df = pd.read_csv('./input/data/eval/info.csv')

## Dataset & DataLoader

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, path, label, transform):
        img_list = []
        for p in tqdm(path):
            img = cv2.imread(p)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_list.append(img)
        
        self.X = img_list
        self.y = label
        self.transform = transform

    def __len__(self):
        len_dataset = len(self.X)
        return len_dataset

    def __getitem__(self, idx):
        X,y = self.X[idx], self.y[idx]
        X = self.transform(image=X)['image']
        return X, y

In [ ]:
class TestDataset(Dataset):
    def __init__(self, path, label, transform):
        img_list = []
        for p in tqdm(path):
            img = cv2.imread(p)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_list.append(img)
        
        self.X = img_list
        self.y = label
        self.transform = transform

    def __len__(self):
        len_dataset = len(self.X)
        return len_dataset

    def __getitem__(self, idx):
        X,y = self.X[idx], self.y[idx]
        X = self.transform(image=X)['image']
        return X

In [ ]:
def get_dataset(df, transform, train=True):
    if train:
        dataset = TrainDataset(path=df['full_path'].values,
                               label=df['label'].values,
                               transform=transform)
    else:
        dataset = TestDataset(path=df['full_path'].values,
                              label=df['ans'].values,
                              transform=transform)
    return dataset

def get_loader(dataset, config, shuffle=True):
    loader = DataLoader(dataset, batch_size=config['BATCH_SIZE'], shuffle=shuffle, 
                        num_workers=config['NUM_WORKERS'], pin_memory=True)
    return loader

In [ ]:
dataset_train = get_dataset(train_df, train_transform, train=True)
dataset_valid = get_dataset(valid_df, test_transform, train=True)

train_dataloader = get_loader(dataset_train, config, shuffle=True)
valid_dataloader = get_loader(dataset_valid, config, shuffle=False)